In [ ]:
PLAYER_NAME = ""                    # Player of interest
ENGINE_NAME = "stockfish"           # Engine for single engine analysis
ENGINES = [ENGINE_NAME]             # Engines for which to retrieve game analyses
QUANTILE = 0.05                     # Best and worst Q% of games
R_MIN = 800                         # Floor for the worst performances
R_MAX = 2300                        # Ceiling for the best performances
EPSILON = 0.01                      # How close can get worst games get to R_min + epsilon*(R_max - R_min)

In [ ]:
%run database.ipynb
%run analysis.ipynb
%run export.ipynb

1. Generate the ACPL data set for the specified player and engines
2. Extract the ACPL values for the single engine of interest
3. Determine the best and worst games
4. Calculate K

In [ ]:
import numpy as np

# Generate the ACPL data set
connection = connect()
player_acpl_df = generate_acpl_for_player(connection, PLAYER_NAME, ENGINES)

# Extract the ACPL values for the engine of interest
df = player_acpl_df[player_acpl_df["engine"] == ENGINE_NAME].copy()

# Get the best and worst Q% of games
ACPL_best = df["acpl"].quantile(QUANTILE)
ACPL_worst = df["acpl"].quantile(1.00 - QUANTILE)

# K defines how quickly Elo drops as ACPL increases
K = np.log(1/EPSILON) / (ACPL_worst - ACPL_best)

print(f'ACPL(best) is {ACPL_best}')
print(f"K for games played by {PLAYER_NAME} and anlysed using {ENGINE_NAME} is {K}")

Having calculated "k" and determined "ACPL_best", the function personal_elo_engine() provides a way of estimating ELO for a game with ACPL equal to "acpl"

In [ ]:
def personal_elo_single_engine(acpl, ACPL_best=ACPL_best):
    return R_MIN + (R_MAX - R_MIN) * np.exp(-K * (acpl - ACPL_best))

df["elo_personal_single"] = personal_elo_single_engine(df["acpl"])
display(df)

A smoothed density plot shows the structure of ACPL values more clearly than a histogram and helping to motivate the exponential decay model

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde

# ACPL values
acpl = df["acpl"].values

# KDE
kde = gaussian_kde(acpl)

x = np.linspace(acpl.min(), acpl.max(), 400)
y = kde(x)

plt.figure(figsize=(8, 5))
plt.plot(x, y, linewidth=2)
plt.xlabel("ACPL")
plt.ylabel("Density")
plt.title(f"{ENGINE_NAME} ACPL Distribution")
plt.show()

# 

In [ ]:
import matplotlib.pyplot as plt

acpl_range = np.linspace(df["acpl"].min(), df["acpl"].max(), 200)
elo_curve = personal_elo_single_engine(acpl_range)

plt.figure(figsize=(8, 5))
plt.scatter(df["acpl"], df["elo_personal_single"], alpha=0.3, label="Games")
plt.plot(acpl_range, elo_curve, linewidth=2, label="Fitted curve")
plt.xlabel(f"{ENGINE_NAME} ACPL")
plt.ylabel("Personal Elo estimate")
plt.title(f"Personal ACPL → Elo model ({ENGINE_NAME} only)")
plt.legend()
plt.show()